# Stocks vs Housing

## Introduction


# Data Sets

The data is contained in three files(from 1980 up to 2019):

* COMP.csv - S&P Composite (https://www.quandl.com/data/YALE/SPCOMP-S-P-Composite)

Here is the schema and explanation of each variable in the files:

**COMP.csv**
* Year (object) - record description (ie transaction, offer received, offer viewed, etc.)
* S & P Composite

# Import Packages

In [2]:
#import holy grail of data science
import pandas as pd
import numpy as np

#more imports
import math
import json
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from tqdm import tqdm


#matplot magic
%matplotlib inline


# Load Data

In [3]:
COMP = pd.read_csv('data/COMP.csv')

# 1.0 Exploratory Data Analysis

## 1.1 COMP

In [4]:
COMP

,Year,S&P Composite,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
0,2019-12-31,3223.380000,NaN,NaN,257.139000,1.910000,3223.380000,NaN,NaN,30.913600
1,2019-11-30,3104.904500,NaN,NaN,257.208000,1.810000,3104.071562,NaN,NaN,29.911395
2,2019-10-31,2977.680000,NaN,NaN,257.346000,1.710000,2975.284860,NaN,NaN,28.840889
3,2019-09-30,2982.156000,57.220000,132.950000,256.759000,1.700000,2986.569552,57.304685,133.146764,29.229401
4,2019-08-31,2897.498182,56.839092,133.723333,256.558000,1.630000,2904.059842,56.967809,134.026163,28.705358
...,...,...,...,...,...,...,...,...,...,...
1783,1871-05-31,4.860000,0.260000,0.400000,12.273812,5.333333,101.818048,5.447056,8.380086,NaN
1784,1871-04-30,4.740000,0.260000,0.400000,12.559226,5.330000,97.047287,5.323269,8.189645,NaN
1785,1871-03-31,4.610000,0.260000,0.400000,13.034972,5.326667,90.940801,5.128982,7.890742,NaN
1786,1871-02-28,4.500000,0.260000,0.400000,12.844641,5.323333,90.086245,5.204983,8.007666,NaN


In [5]:
COMP = COMP[['Year', 'S&P Composite']]

In [6]:
#get shape
COMP.shape

(1788, 2)

In [7]:
# data types
COMP.dtypes

Year              object
S&P Composite    float64
dtype: object

In [8]:
#quick stats
COMP.describe()

,S&P Composite
count,1788.000000
mean,287.415617
std,579.830261
min,2.730000
25%,7.847500
50%,16.915000
75%,134.725000
max,3223.380000


In [9]:
COMP.corr()

,S&P Composite
S&P Composite,1.0


In [10]:
#check for missing values if any
COMP.isna().mean()

Year             0.0
S&P Composite    0.0
dtype: float64

In [17]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

fig = px.line(COMP, x='Year', y='S&P Composite', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [19]:
COMP['Date'] = pd.to_datetime(COMP.Year)


/Users/pierre/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
COMP['Year Composite']= COMP.groupby(COMP.Date.dt.year)['S&P Composite'].transform('mean')

/Users/pierre/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
COMP.head()

,Year,S&P Composite,Date,Year Index,Year Composite
0,2019-12-31,3223.380000,2019-12-31,2916.38686,2916.38686
1,2019-11-30,3104.904500,2019-11-30,2916.38686,2916.38686
2,2019-10-31,2977.680000,2019-10-31,2916.38686,2916.38686
3,2019-09-30,2982.156000,2019-09-30,2916.38686,2916.38686
4,2019-08-31,2897.498182,2019-08-31,2916.38686,2916.38686


In [26]:
COMP = COMP.drop(['Year','Year Index','S&P Composite'], axis=1)

In [30]:
COMP['Date'] = pd.DatetimeIndex(COMP['Date']).year

In [31]:
# dropping ALL duplicte values 
COMP.drop_duplicates(subset ="Date", 
                     keep = "first", inplace = True) 
  

In [32]:
COMP

,Date,Year Composite
0,2019,2916.386860
12,2018,2744.680000
24,2017,2448.217500
36,2016,2091.844167
48,2015,2061.199167
...,...,...
1728,1875,4.447500
1740,1874,4.570000
1752,1873,4.801667
1764,1872,5.029167


In [33]:
COMP.to_csv('data/COMPclean.csv', index=False)